In [ ]:
# default_exp core
%load_ext lab_black

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import argparse
import pandas as pd
from pathlib import Path

if hasattr(__builtins__, "__IPYTHON__") or __name__ != "__main__":
    file = "sites.csv"
else:
    aparser = argparse.ArgumentParser()
    add_arg = aparser.add_argument
    add_arg("-f", "--file", required=True)
    args = aparser.parse_args()
    file = args.file

file_obj = Path(file)
df = pd.read_csv(file_obj, delimiter="|")
df = df.applymap(lambda x: x.strip())
df.columns = [x.strip() for x in df.columns]

In [ ]:
for index, series in df.iterrows():
    row = series.to_dict()
    print(row)
    if index > 2:
        break

{'site': 'GuerillaTech.com', 'title': 'Guerilla Tech', 'gaid': 'G-N4RYB5DCV4', 'tagline': 'Low Tech Liability, High Return'}
{'site': 'Levinux.com', 'title': 'Levinux', 'gaid': 'G-0H19QDRNTL', 'tagline': 'A Micro Linux Distribution for Education'}
{'site': 'LinuxPythonvimgit.com', 'title': 'Linux, Python, vim & git', 'gaid': 'G-ZNMVJFLRD2', 'tagline': 'Four Tools for Life'}
{'site': 'LunderVand.com', 'title': 'LunderVand', 'gaid': 'G-TJRX2PSWKT', 'tagline': 'The Story of Technology That Must Be Told'}


In [ ]:
!nbdev_build_lib

Converted 00_core.ipynb.
Converted index.ipynb.
